# Applied Quantum Computing Lab: Variational Quantum Eigensolver (VQE) - SOLUTIONS

The Variational Quantum Eigensolver (VQE) is a hybrid quantum-classical algorithm designed to find the ground state energy of a molecule or a Hamiltonian system. It's one of the most promising algorithms for near-term quantum computers due to its ability to work with noisy, limited-resource quantum hardware.

## Prerequisites
- Basic understanding of quantum gates and circuits
- Familiarity with Python and Qiskit
- Basic knowledge of linear algebra and quantum mechanics

## Learning Objectives
By completing this lab, you will:
1. Understand the principles of the VQE algorithm
2. Implement a VQE circuit for a simple Hamiltonian
3. Optimize the quantum circuit parameters using classical optimization
4. Analyze the results and understand the quantum advantage in chemistry applications

## Introduction

### How VQE Works

1. **Prepare a parameterized quantum state**: Create a quantum circuit with adjustable parameters
2. **Measure the energy**: Use the quantum computer to measure the expectation value of the Hamiltonian
3. **Update parameters**: Use a classical optimizer to find parameters that minimize the energy
4. **Iterate**: Repeat until convergence

In this lab, we'll implement VQE to find the ground state energy of a simple Hamiltonian using Qiskit 2.


In [ ]:
# Import required libraries
import numpy as np
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
from qiskit.quantum_info import SparsePauliOp

# For visualization
import matplotlib.pyplot as plt
%matplotlib inline

## Exercise 1: Creating a Hamiltonian

The Hamiltonian is the energy operator of our system. For this example, we'll use a simple 2-qubit Hamiltonian that represents the interactions in a toy molecular system. In Qiskit 2, we can represent this using `SparsePauliOp`.

In [ ]:
# Define a simple 2-qubit Hamiltonian
# H = Z₀⊗I₁ + I₀⊗Z₁ + 0.5 X₀⊗X₁
# This represents a simplified Ising model

# Create the Hamiltonian using SparsePauliOp
hamiltonian = SparsePauliOp(
    ['ZI', 'IZ', 'XX'],  # Pauli strings
    [1.0, 1.0, 0.5]      # Coefficients
)

print("Hamiltonian:")
print(hamiltonian)

## Exercise 2: Variational Form (Ansatz)

The ansatz is a parameterized quantum circuit that prepares a trial state. The goal is to find the parameters that minimize the energy. We'll use a simple ansatz with rotation gates and entangling operations.

In [ ]:
# Create a parameterized ansatz with rotation gates and entanglement
def create_ansatz(parameters):
    """Create a parameterized quantum circuit.
    
    Args:
        parameters (list): List of 6 parameters for the circuit
    
    Returns:
        QuantumCircuit: The parameterized quantum circuit
    """
    qc = QuantumCircuit(2)
    
    # First layer of rotations
    qc.rx(parameters[0], 0)
    qc.ry(parameters[1], 0)
    qc.rx(parameters[2], 1)
    qc.ry(parameters[3], 1)
    
    # Entangling layer
    qc.cx(0, 1)
    
    # Second layer of rotations
    qc.rx(parameters[4], 0)
    qc.rx(parameters[5], 1)
    
    return qc

# Visualize the ansatz with some random parameters
initial_params = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6]
ansatz = create_ansatz(initial_params)
print("Sample ansatz circuit:")
print(ansatz.draw())

## Exercise 3: Energy Calculation

For each set of parameters, we need to calculate the expectation value of the Hamiltonian. This is done by preparing the state defined by our ansatz and measuring the energy using our Hamiltonian.

In [ ]:
# Function to calculate the expectation value of the Hamiltonian
def calculate_energy(parameters):
    """Calculate the expectation value of the Hamiltonian for given parameters.
    
    Args:
        parameters (list): Ansatz parameters
    
    Returns:
        float: Expected energy
    """
    # Create the ansatz circuit with current parameters
    qc = create_ansatz(parameters)
    
    # Configure the simulator to return the statevector
    simulator = AerSimulator(method='statevector')
    
    # Save the statevector state
    qc.save_statevector()
    
    # Run the circuit and get the statevector
    job = simulator.run(qc)
    result = job.result()
    
    # Get the statevector from the result
    state = result.get_statevector()
    
    # Calculate the expectation value of the Hamiltonian
    energy = state.expectation_value(hamiltonian).real
    
    return energy

# Test the energy calculation with initial parameters
energy = calculate_energy(initial_params)
print(f"Energy with initial parameters: {energy:.6f}")

## Exercise 4: Classical Optimization

Now we'll use a classical optimizer to find the parameters that minimize the energy. We'll use the COBYLA optimizer from scipy, which is a gradient-free optimization method.

In [ ]:
# Perform the optimization using scipy's minimize function
from scipy.optimize import minimize

# Function to minimize
def objective_function(parameters):
    return calculate_energy(parameters)

# Initial parameters
initial_params = np.random.random(6) * 2 * np.pi

# Keep track of energy values during optimization
energy_values = []

# Callback function to track optimization progress
def callback(params):
    energy = objective_function(params)
    energy_values.append(energy)
    if len(energy_values) % 5 == 0:  # Print every 5 iterations
        print(f"Iteration {len(energy_values)}: Energy = {energy:.6f}")

# Run the optimizer
print("Starting optimization...")
result = minimize(
    objective_function,
    initial_params,
    method='COBYLA',
    callback=callback,
    options={'maxiter': 100}
)

# Print results
print("\nOptimization complete!")
print(f"Final energy: {result.fun:.6f}")
print(f"Optimal parameters: {result.x}")

## Exercise 5: Results Visualization

Let's visualize the convergence of the energy during the optimization process.

In [ ]:
# Plot the energy convergence
plt.figure(figsize=(10, 6))
plt.plot(energy_values, 'o-')
plt.xlabel('Iteration')
plt.ylabel('Energy')
plt.title('VQE Energy Convergence')
plt.grid(True)
plt.show()

## Exercise 6: Final Optimized Circuit

Let's visualize the circuit with the optimized parameters and calculate the probabilities of measuring different states.

In [ ]:
# Create and display the final circuit with optimized parameters
final_circuit = create_ansatz(result.x)
print("Final optimized circuit:")
print(final_circuit.draw())

# Add measurements to get the state probabilities
final_circuit_with_measure = create_ansatz(result.x)
final_circuit_with_measure.measure_all()

# Run the circuit
simulator = AerSimulator()
result_counts = simulator.run(final_circuit_with_measure, shots=1000).result().get_counts()

# Display the results
print("\nState probabilities from 1000 measurements:")
for state, counts in result_counts.items():
    print(f"State |{state}>: {counts/1000:.4f} probability")

# Plot the histogram
plt.figure(figsize=(10, 6))
states = result_counts.keys()
probabilities = [result_counts[state]/1000 for state in states]
plt.bar(states, probabilities)
plt.xlabel('State')
plt.ylabel('Probability')
plt.title('Measurement Probabilities of the Ground State')
plt.ylim(0, 1)
plt.show()

## Exercise 7: Analyzing the Ground State

Let's analyze the ground state that we've found through the VQE algorithm.

In [ ]:
# Calculate the statevector of the optimized circuit (without measurements)
final_circuit = create_ansatz(result.x)
final_circuit.save_statevector()
simulator = AerSimulator(method='statevector')
job = simulator.run(final_circuit)
result_state = job.result().get_statevector()

# Print the statevector elements and their probabilities
print("Final statevector:")
for i, amplitude in enumerate(result_state):
    state = bin(i)[2:].zfill(2)
    probability = abs(amplitude)**2
    print(f"|{state}>: amplitude = {amplitude:.4f}, probability = {probability:.4f}")

# For our simple Hamiltonian, the ground state can be calculated analytically
# For the Hamiltonian H = Z₀⊗I₁ + I₀⊗Z₁ + 0.5 X₀⊗X₁, 
# the ground state depends on the coefficients, but is often a superposition
# of computational basis states

# For demonstration, we'll calculate the energy of our state explicitly
from qiskit.quantum_info import Statevector

# Calculate energy by components
print("\nEnergy components:")
for pauli_string, coeff in zip(hamiltonian.paulis, hamiltonian.coeffs):
    component_value = result_state.expectation_value(SparsePauliOp([pauli_string], [1.0])).real
    energy_contribution = coeff * component_value
    print(f"{pauli_string}: {coeff} × {component_value:.6f} = {energy_contribution:.6f}")

print(f"\nTotal energy: {calculate_energy(result.x):.6f}")

# We can also compare with the theoretical minimum eigenvalue
# For this simple Hamiltonian, we can calculate it analytically
# For a general Hamiltonian, we would need to diagonalize it
print("\nTheoretical minimum eigenvalue calculation:")
from numpy import linalg as LA

# Convert Hamiltonian to matrix form (for 2 qubits)
hamiltonian_matrix = hamiltonian.to_matrix()
print("Hamiltonian matrix:")
print(hamiltonian_matrix)

# Calculate eigenvalues and eigenvectors
eigenvalues, eigenvectors = LA.eigh(hamiltonian_matrix)
print(f"Eigenvalues: {eigenvalues}")

# The ground state is the eigenvector corresponding to the smallest eigenvalue
theoretical_ground_state = eigenvectors[:, 0]
print("Theoretical ground state vector:")
for i, amplitude in enumerate(theoretical_ground_state):
    state = bin(i)[2:].zfill(2)
    print(f"|{state}>: {amplitude}")

# Calculate fidelity between our VQE result and the theoretical ground state
fidelity = abs(np.vdot(theoretical_ground_state, result_state.data))**2
print(f"\nFidelity between VQE state and theoretical ground state: {fidelity:.6f}")
print(f"VQE energy: {calculate_energy(result.x):.6f}")
print(f"Theoretical minimum energy: {eigenvalues[0]:.6f}")
print(f"Energy difference: {abs(calculate_energy(result.x) - eigenvalues[0]):.6f}")

## Challenge Exercise: Different Ansatz Structures

Try implementing different ansatz structures and compare their performance:

1. A simpler ansatz with fewer parameters
2. A more complex ansatz with more entangling gates
3. A hardware-efficient ansatz that follows a specific topology

How does the choice of ansatz affect the final energy and the optimization process?

In [ ]:
# Implementation of different ansatz designs

# 1. Simpler ansatz with fewer parameters
def create_simple_ansatz(parameters):
    """Create a simpler ansatz with only 3 parameters."""
    qc = QuantumCircuit(2)
    
    # Only one rotation per qubit
    qc.rx(parameters[0], 0)
    qc.rx(parameters[1], 1)
    
    # Entangling gate
    qc.cx(0, 1)
    
    # One more rotation
    qc.rx(parameters[2], 0)
    
    return qc

# 2. More complex ansatz with more entangling gates
def create_complex_ansatz(parameters):
    """Create a more complex ansatz with 10 parameters and more entanglement."""
    qc = QuantumCircuit(2)
    
    # First layer
    qc.rx(parameters[0], 0)
    qc.ry(parameters[1], 0)
    qc.rx(parameters[2], 1)
    qc.ry(parameters[3], 1)
    
    # First entangling layer
    qc.cx(0, 1)
    
    # Second layer
    qc.rx(parameters[4], 0)
    qc.ry(parameters[5], 0)
    qc.rx(parameters[6], 1)
    qc.ry(parameters[7], 1)
    
    # Second entangling layer (reverse direction)
    qc.cx(1, 0)
    
    # Final rotations
    qc.rx(parameters[8], 0)
    qc.rx(parameters[9], 1)
    
    return qc

# 3. Hardware-efficient ansatz (linear topology)
def create_hardware_efficient_ansatz(parameters):
    """Create a hardware-efficient ansatz for linear qubit topology."""
    qc = QuantumCircuit(2)
    
    # First layer of rotations
    qc.rx(parameters[0], 0)
    qc.rz(parameters[1], 0)
    qc.rx(parameters[2], 1)
    qc.rz(parameters[3], 1)
    
    # Entangling layer (only nearest-neighbor)
    qc.cx(0, 1)
    
    # Second layer of rotations
    qc.rx(parameters[4], 0)
    qc.rz(parameters[5], 0)
    qc.rx(parameters[6], 1)
    qc.rz(parameters[7], 1)
    
    return qc

# Function to run VQE with a given ansatz
def run_vqe(ansatz_function, num_parameters):
    """Run VQE with the specified ansatz."""
    # Create an energy calculation function for this ansatz
    def calc_energy(params):
        qc = ansatz_function(params)
        qc.save_statevector()
        simulator = AerSimulator(method='statevector')
        job = simulator.run(qc)
        state = job.result().get_statevector()
        return state.expectation_value(hamiltonian).real
    
    # Run the optimization
    init_params = np.random.random(num_parameters) * 2 * np.pi
    energies = []
    
    def callback(params):
        energy = calc_energy(params)
        energies.append(energy)
    
    result = minimize(calc_energy, init_params, method='COBYLA', callback=callback, options={'maxiter': 100})
    
    return result.fun, result.x, energies

# Compare the three ansatz designs
print("Comparing different ansatz designs:")

# Simple ansatz (3 parameters)
simple_energy, simple_params, simple_energies = run_vqe(create_simple_ansatz, 3)
print(f"Simple ansatz final energy: {simple_energy:.6f}")

# Standard ansatz (6 parameters, from our original implementation)
standard_energy, standard_params, standard_energies = run_vqe(create_ansatz, 6)
print(f"Standard ansatz final energy: {standard_energy:.6f}")

# Complex ansatz (10 parameters)
complex_energy, complex_params, complex_energies = run_vqe(create_complex_ansatz, 10)
print(f"Complex ansatz final energy: {complex_energy:.6f}")

# Hardware-efficient ansatz (8 parameters)
hw_energy, hw_params, hw_energies = run_vqe(create_hardware_efficient_ansatz, 8)
print(f"Hardware-efficient ansatz final energy: {hw_energy:.6f}")

# Calculate the theoretical minimum energy for reference
eigenvalues = LA.eigvalsh(hamiltonian.to_matrix())
min_energy = min(eigenvalues)
print(f"Theoretical minimum energy: {min_energy:.6f}")

# Plot convergence comparison
plt.figure(figsize=(12, 8))
plt.plot(simple_energies, label='Simple (3 params)')
plt.plot(standard_energies, label='Standard (6 params)')
plt.plot(complex_energies, label='Complex (10 params)')
plt.plot(hw_energies, label='Hardware-efficient (8 params)')
plt.axhline(y=min_energy, color='r', linestyle='--', label='Theoretical minimum')
plt.xlabel('Iteration')
plt.ylabel('Energy')
plt.title('Convergence Comparison of Different Ansatz Designs')
plt.legend()
plt.grid(True)
plt.show()

# Print final energy difference from theoretical minimum
print("\nEnergy difference from theoretical minimum:")
print(f"Simple ansatz: {abs(simple_energy - min_energy):.6f}")
print(f"Standard ansatz: {abs(standard_energy - min_energy):.6f}")
print(f"Complex ansatz: {abs(complex_energy - min_energy):.6f}")
print(f"Hardware-efficient ansatz: {abs(hw_energy - min_energy):.6f}")

# Analyze the results
print("\nAnalysis:")
print("1. The simpler ansatz with fewer parameters is less expressive but may converge faster.")
print("2. The complex ansatz with more parameters can potentially find better solutions but may require more iterations.")
print("3. The hardware-efficient ansatz is designed to work well on actual quantum hardware with specific connectivity.")

## Reflection Questions

1. How does the VQE algorithm combine quantum and classical computing? What is the advantage of this hybrid approach?

2. Why is VQE particularly well-suited for near-term quantum computers with limited qubits and relatively high noise?

3. How would the number of required qubits scale for simulating real molecular systems compared to classical computational chemistry methods?

4. What are some practical challenges in scaling VQE to larger molecular systems?

## Reflection Answers

1. **How does VQE combine quantum and classical computing?**

   VQE leverages a hybrid approach where:
   - The quantum computer prepares and measures quantum states (trial wavefunctions)
   - The classical computer processes measurement results and optimizes circuit parameters
   
   This hybrid approach offers several advantages:
   - It offloads the parameter optimization to classical computers, which are well-suited for this task
   - It only requires shallow quantum circuits that run for short durations, making it feasible on noisy quantum hardware
   - The quantum computer is used specifically for what it does best: representing and manipulating quantum states that would require exponential resources classically
   - The iterative nature allows for error mitigation techniques to be applied

2. **Why is VQE well-suited for near-term quantum computers?**

   VQE is well-suited for Noisy Intermediate-Scale Quantum (NISQ) devices because:
   - It requires relatively shallow circuits, reducing the impact of decoherence
   - It's robust against certain types of errors due to the variational nature (parameters adjust to compensate for systematic errors)
   - It doesn't require quantum error correction, which would demand significantly more qubits
   - The hybrid approach means that most of the computational burden is handled by classical computers
   - The algorithm can be adapted to hardware constraints by modifying the ansatz design
   - Results improve incrementally, providing useful approximations even before reaching the exact ground state

3. **Qubit scaling for molecular simulations**

   For simulating molecular systems:
   - Classical methods often scale exponentially with molecule size, becoming intractable for larger molecules
   - Quantum VQE requires qubits that scale linearly with the number of molecular orbitals
   - Typically, simulating a molecule with N electronic orbitals requires approximately 2N qubits (with certain encoding techniques)
   - With qubit-reduction techniques like symmetry considerations and active space selection, the number can be reduced further
   
   This provides a potentially exponential advantage over classical methods for larger molecules, especially when high accuracy is required. For example, simulating the FeMoCo enzyme classically would require more memory than atoms in the observable universe, but could theoretically be done on a quantum computer with a few hundred logical qubits.

4. **Practical challenges in scaling VQE**

   Several challenges exist in scaling VQE to larger systems:
   - **Circuit depth**: Larger molecules require deeper circuits, increasing susceptibility to noise
   - **Parameter optimization**: The parameter landscape becomes more complex and harder to optimize as the number of parameters grows
   - **Measurement overhead**: More terms in the Hamiltonian mean more measurements are needed
   - **Error rates**: Current quantum hardware error rates limit the achievable accuracy
   - **Barren plateaus**: Gradients can vanish exponentially with system size in certain ansatz designs
   - **Hamiltonian representation**: The number of Pauli terms in the Hamiltonian grows rapidly with system size
   - **Classical preprocessing**: Computing molecular integrals classically becomes challenging for large molecules
   - **Hardware connectivity**: Limited qubit connectivity on real devices constrains the implementable ansatz designs

## Conclusion

In this lab, you've implemented the VQE algorithm to find the ground state energy of a simple 2-qubit Hamiltonian. The key steps included:

1. Defining a Hamiltonian representing our system
2. Creating a parameterized quantum circuit (ansatz)
3. Calculating the energy expectation value for a given set of parameters
4. Using classical optimization to find the parameters that minimize the energy
5. Analyzing and visualizing the results

VQE is particularly important for quantum chemistry applications, where it can be used to calculate molecular properties more efficiently than classical methods for complex molecules. As quantum hardware improves, VQE can be applied to increasingly complex systems.

## References

1. Peruzzo, A. et al. (2014). A variational eigenvalue solver on a photonic quantum processor. Nature Communications, 5, 4213.
2. [Qiskit Documentation on VQE](https://qiskit.org/documentation/tutorials/algorithms/03_vqe.html)
3. McArdle, S. et al. (2020). Quantum computational chemistry. Reviews of Modern Physics, 92(1), 015003.